In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
batchsize = 8

In [5]:
train_data_gen = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range = 0.2, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    validation_split = 0.2
)

In [6]:
train_data = train_data_gen.flow_from_directory(
    r'C:\\Capstone_django-version\\Drowsiness_Detection_System\\templates\\Prepared_Data\\train', 
    target_size = (80, 80), 
    batch_size = batchsize, 
    class_mode = 'categorical',
    subset = 'training'
)

validation_data = train_data_gen.flow_from_directory(
    r'C:\\Capstone_django-version\\Drowsiness_Detection_System\\templates\\Prepared_Data\\train', 
    target_size = (80, 80), 
    batch_size = batchsize, 
    class_mode = 'categorical',
    subset = 'validation'
)

Found 64687 images belonging to 2 classes.
Found 16171 images belonging to 2 classes.


In [7]:
test_data_gen = ImageDataGenerator(rescale = 1./255)

test_data = test_data_gen.flow_from_directory(
    r'C:\\Capstone_django-version\\Drowsiness_Detection_System\\templates\\Prepared_Data\\test', 
    target_size = (80, 80), 
    batch_size = batchsize, 
    class_mode = 'categorical',
)

Found 4040 images belonging to 2 classes.


In [8]:
base_model = ResNet50(
    include_top = False,
    weights = 'imagenet',
    input_tensor = Input(shape = (80, 80, 3))
)

had_model = base_model.output

In [9]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [10]:
had_model = Flatten()(had_model)
had_model = Dense(64, activation = 'relu')(had_model)
had_model = Dropout(0.5)(had_model)
had_model = Dense(2, activation = 'softmax')(had_model)

model = Model(inputs = base_model.input, outputs = had_model)
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [13]:
checkpoint = ModelCheckpoint(r'C:\\Capstone_django-version\\Drowsiness_Detection_System\\templates\\models\\model.h5', monitor = 'val_loss', save_best_only = True, verbose = 3)
# verbose is like the font-size
# save_the_best takes that model only whose validation loss is very less by monitoring the val_loss

earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, verbose = 3, restore_best_weights = True)
# We use the EarlyStopping if we are waiting for long but either the accuracy is not increasing or validation loss is not decreasing than also.
# patience = 7, means it will have the patience upto 7 epochs, if the accuracy is not increasing or the validation loss is not decreasing, the model will stop early without completing all the epochs.

learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 3)

callbacks = [checkpoint, earlystop, learning_rate]

In [15]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit_generator(
    train_data,
    steps_per_epoch = train_data.samples//batchsize,
    validation_data = validation_data,
    validation_steps = validation_data.samples//batchsize,
    callbacks = callbacks,
    epochs = 16
)

Epoch 1/16


C:\Users\Arshdeep\AppData\Local\Temp\ipykernel_4928\789821392.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


 249/8085 [..............................] - ETA: 22:20 - loss: 0.6932 - accuracy: 0.5090

KeyboardInterrupt: 

In [16]:
train_loss, train_accuracy = model.evaluate_generator(train_data)
print(train_accuracy)
print(train_loss)

C:\Users\Arshdeep\AppData\Local\Temp\ipykernel_4928\4054689642.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  train_loss, train_accuracy = model.evaluate_generator(train_data)


0.5061758756637573
0.6930851936340332


In [19]:
validation_loss, validation_accuracy = model.evaluate_generator(validation_data)
print(validation_accuracy)
print(validation_loss)

C:\Users\Arshdeep\AppData\Local\Temp\ipykernel_4928\2308148602.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  validation_loss, validation_accuracy = model.evaluate_generator(validation_data)


0.5061529874801636
0.693079948425293


In [20]:
test_loss, test_accuracy = model.evaluate_generator(test_data)
print(test_accuracy)
print(test_loss)

C:\Users\Arshdeep\AppData\Local\Temp\ipykernel_4928\3362072667.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(test_data)


0.500990092754364
0.6932429671287537
